In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/ransomware/'

In [27]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

Found 101 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(101, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.1)

In [5]:
X_train.shape

(90, 64, 64, 3)

In [6]:
X_test.shape

(11, 64, 64, 3)

In [7]:
y_train.shape

(90, 10)

In [8]:
y_test.shape

(11, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [26]:
Malware_model = malware_model()

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(90, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([9, 5, 8, 3, 8, 2, 5, 2, 6, 3, 8, 0, 3, 7, 7, 3, 2, 8, 8, 6, 4, 4,
       4, 9, 4, 9, 6, 7, 6, 3, 2, 3, 7, 0, 3, 5, 8, 0, 2, 4, 5, 8, 4, 9,
       4, 3, 7, 1, 6, 0, 1, 2, 0, 1, 5, 3, 4, 7, 9, 5, 6, 6, 4, 2, 0, 1,
       8, 7, 9, 0, 7, 6, 2, 5, 8, 5, 1, 6, 0, 9, 8, 1, 4, 1, 8, 1, 2, 1,
       9, 9])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

Epoch 1/100


2023-05-02 16:59:10.302755: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


3/3 [==============================] - 3s 513ms/step - loss: 0.6822 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4889 - val_loss: 0.5343 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5298
Epoch 2/100
3/3 [==============================] - 0s 72ms/step - loss: 0.5489 - f1_m: 0.0123 - precision_m: 0.1667 - recall_m: 0.0064 - auc: 0.4848 - val_loss: 0.4371 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5289
Epoch 3/100
3/3 [==============================] - 0s 71ms/step - loss: 0.5063 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4607 - val_loss: 0.3881 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5014
Epoch 4/100
3/3 [==============================] - 0s 69ms/step - loss: 0.4312 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5284 - val_loss: 0.3652 - val_f1_

Epoch 32/100
3/3 [==============================] - 0s 95ms/step - loss: 0.1442 - f1_m: 0.6809 - precision_m: 0.8433 - recall_m: 0.5713 - auc: 0.9733 - val_loss: 0.1654 - val_f1_m: 0.7059 - val_precision_m: 1.0000 - val_recall_m: 0.5455 - val_auc: 0.9573
Epoch 33/100
3/3 [==============================] - 0s 97ms/step - loss: 0.1264 - f1_m: 0.8351 - precision_m: 0.9567 - recall_m: 0.7431 - auc: 0.9874 - val_loss: 0.1626 - val_f1_m: 0.7778 - val_precision_m: 1.0000 - val_recall_m: 0.6364 - val_auc: 0.9486
Epoch 34/100
3/3 [==============================] - 0s 99ms/step - loss: 0.1271 - f1_m: 0.8058 - precision_m: 0.9158 - recall_m: 0.7195 - auc: 0.9842 - val_loss: 0.1616 - val_f1_m: 0.7778 - val_precision_m: 1.0000 - val_recall_m: 0.6364 - val_auc: 0.9467
Epoch 35/100
3/3 [==============================] - 0s 93ms/step - loss: 0.1289 - f1_m: 0.7158 - precision_m: 0.9015 - recall_m: 0.5970 - auc: 0.9819 - val_loss: 0.1518 - val_f1_m: 0.7778 - val_precision_m: 1.0000 - val_recall_m: 0.636

3/3 [==============================] - 0s 63ms/step - loss: 0.0223 - f1_m: 0.9831 - precision_m: 0.9831 - recall_m: 0.9831 - auc: 0.9999 - val_loss: 0.1371 - val_f1_m: 0.8571 - val_precision_m: 0.9000 - val_recall_m: 0.8182 - val_auc: 0.9738
Epoch 65/100
3/3 [==============================] - 0s 56ms/step - loss: 0.0231 - f1_m: 0.9775 - precision_m: 0.9906 - recall_m: 0.9648 - auc: 0.9999 - val_loss: 0.1433 - val_f1_m: 0.8571 - val_precision_m: 0.9000 - val_recall_m: 0.8182 - val_auc: 0.9706
Epoch 66/100
3/3 [==============================] - 0s 57ms/step - loss: 0.0229 - f1_m: 0.9936 - precision_m: 0.9936 - recall_m: 0.9936 - auc: 1.0000 - val_loss: 0.1573 - val_f1_m: 0.8571 - val_precision_m: 0.9000 - val_recall_m: 0.8182 - val_auc: 0.9770
Epoch 67/100
3/3 [==============================] - 0s 55ms/step - loss: 0.0210 - f1_m: 0.9936 - precision_m: 0.9936 - recall_m: 0.9936 - auc: 1.0000 - val_loss: 0.1652 - val_f1_m: 0.8571 - val_precision_m: 0.9000 - val_recall_m: 0.8182 - val_auc: 

3/3 [==============================] - 0s 54ms/step - loss: 0.0127 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - auc: 1.0000 - val_loss: 0.1544 - val_f1_m: 0.8182 - val_precision_m: 0.8182 - val_recall_m: 0.8182 - val_auc: 0.9775
Epoch 97/100
3/3 [==============================] - 0s 54ms/step - loss: 0.0123 - f1_m: 0.9933 - precision_m: 1.0000 - recall_m: 0.9872 - auc: 1.0000 - val_loss: 0.1609 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 0.9752
Epoch 98/100
3/3 [==============================] - 0s 54ms/step - loss: 0.0163 - f1_m: 0.9845 - precision_m: 0.9845 - recall_m: 0.9845 - auc: 0.9999 - val_loss: 0.1495 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 0.9761
Epoch 99/100
3/3 [==============================] - 0s 57ms/step - loss: 0.0103 - f1_m: 0.9967 - precision_m: 1.0000 - recall_m: 0.9936 - auc: 1.0000 - val_loss: 0.1339 - val_f1_m: 0.7273 - val_precision_m: 0.7273 - val_recall_m: 0.7273 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.1252 - f1_m: 0.8571 - precision_m: 0.9000 - recall_m: 0.8182 - auc: 0.9830


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 19ms/step - loss: 0.1252 - f1_m: 0.8571 - precision_m: 0.9000 - recall_m: 0.8182 - auc: 0.9830


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.8571427464485168
